<a href="https://colab.research.google.com/github/ZeroLeon/NLP_Project_with_FastAI/blob/master/BBC_TopicModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Link](https://github.com/fastai/course-nlp/blob/master/2-svd-nmf-topic-modeling.ipynb) of corresponding fastai NLP course notebook

# Topic Modeling for BBC News





Notes:

1.   Relative word  frequency  in vocab.
2.   The topic words are the most possible words to divide defferent topics apart.
3.   It's very simlilar to the collabrative filtering.
4. With SVD we can actually choose a suitable numbers of topics. 

In [45]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
!kaggle --help

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 67 bytes
usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...

optional arguments:
  -h, --help            show this help message and exit
  -v, --version         show program's version number and exit

commands:
  {competitions,c,datasets,d,kernels,k,config}
                        Use one of:
                        competitions {list, files, download, submit, submissions, leaderboard}
                        datasets {list, files, download, create, version, init, metadata, status}
                        config {view, set, unset}
    competitions        Commands related to Kaggle competitions
    datasets            Commands related to Kaggle datasets
    kernels             Commands related to Kaggle kernels
    config              Configuration settings


In [53]:
!kaggle datasets download -d pariza/bbc-news-summary

  0% 0.00/3.75M [00:00<?, ?B/s]
100% 3.75M/3.75M [00:00<00:00, 61.4MB/s]


In [54]:
!unzip bbc-news-summary.zip

Archive:  bbc-news-summary.zip
  inflating: BBC News Summary.rar    


In [0]:
!unrar e BBC\ News\ Summary.rar

In [0]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import decomposition
from glob import glob
import os

In [0]:
np.set_printoptions(suppress=True)

In [65]:
#ex
pth = '/content/'
print(glob(pth+"*.txt"))

['/content/181.txt', '/content/355.txt', '/content/261.txt', '/content/254.txt', '/content/246.txt', '/content/125.txt', '/content/188.txt', '/content/255.txt', '/content/051.txt', '/content/277.txt', '/content/105.txt', '/content/264.txt', '/content/337.txt', '/content/170.txt', '/content/039.txt', '/content/457.txt', '/content/443.txt', '/content/251.txt', '/content/216.txt', '/content/067.txt', '/content/281.txt', '/content/317.txt', '/content/045.txt', '/content/314.txt', '/content/352.txt', '/content/463.txt', '/content/200.txt', '/content/434.txt', '/content/420.txt', '/content/279.txt', '/content/446.txt', '/content/316.txt', '/content/030.txt', '/content/090.txt', '/content/354.txt', '/content/145.txt', '/content/226.txt', '/content/111.txt', '/content/031.txt', '/content/134.txt', '/content/075.txt', '/content/295.txt', '/content/287.txt', '/content/482.txt', '/content/479.txt', '/content/494.txt', '/content/300.txt', '/content/499.txt', '/content/046.txt', '/content/129.txt',

In [0]:
glob??

In [0]:
filenames_all = []
filenames_all.extend(glob( '/content/'+ '/*.txt'))

In [122]:
len(filenames_all)

511

In [0]:
filenames = filenames_all[:20]

In [69]:
vectorizer = TfidfVectorizer(input='filename', stop_words='english')
dtm = vectorizer.fit_transform(filenames).toarray()
vocab = np.array(vectorizer.get_feature_names())
dtm.shape, len(vocab)

((20, 1276), 1276)

In [70]:
#ex
vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='filename', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [72]:
[f.split("/")[2] for f in filenames]

['181.txt',
 '355.txt',
 '261.txt',
 '254.txt',
 '246.txt',
 '125.txt',
 '188.txt',
 '255.txt',
 '051.txt',
 '277.txt',
 '105.txt',
 '264.txt',
 '337.txt',
 '170.txt',
 '039.txt',
 '457.txt',
 '443.txt',
 '251.txt',
 '216.txt',
 '067.txt']

In [78]:
#ex 
dtm

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.04898601, ..., 0.04049176, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.07168083]])

### NMF

In [0]:
clf = decomposition.NMF(n_components=10, random_state=1)

W1 = clf.fit_transform(dtm)
H1 = clf.components_

In [79]:
#embedding to 10 dimensional matrix 
W1.shape,H1.shape

((20, 10), (10, 1276))

In [0]:
num_top_words=8

def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

In [0]:
def get_all_topic_words(H):
    top_indices = lambda t: {i for i in np.argsort(t)[:-num_top_words-1:-1]}
    topic_indices = [top_indices(t) for t in H]
    return sorted(set.union(*topic_indices))

In [0]:
ind = get_all_topic_words(H1)

In [0]:
#ex 
import pandas as pd

def get_words_df(a):
  top_ws = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
  top_w = ([top_ws(t) for t in a])
  df = pd.DataFrame(top_w)
  return df

In [111]:
get_words_df(H1)


,0,1,2,3,4,5,6,7
0,broadband,bt,million,cable,line,operators,news,uk
1,yahoo,search,google,web,games,tv,speegle,gamers
2,mobile,nokia,flynn,said,mr,romen,phone,cut
3,february,star,kim,battlefront,game,wars,return,clijsters
4,asimo,robot,car,robots,motorsport,left,tansley,gear
5,fbi,virus,mails,attachment,nuclear,inspectors,computer,agency
6,site,bittorrent,apple,bloggers,legal,ruling,host,server
7,wong,software,colour,blind,mr,weather,study,explore
8,women,seed,tuesday,sydney,plays,round,quest,men
9,phone,relay,deaf,premium,language,sign,service,calls


In [77]:
vocab[ind]

array(['agency', 'apple', 'asimo', 'attachment', 'battlefront',
       'bittorrent', 'blind', 'bloggers', 'broadband', 'bt', 'cable',
       'calls', 'car', 'clijsters', 'colour', 'computer', 'cut', 'deaf',
       'explore', 'fbi', 'february', 'flynn', 'game', 'gamers', 'games',
       'gear', 'google', 'host', 'inspectors', 'kim', 'language', 'left',
       'legal', 'line', 'mails', 'men', 'million', 'mobile', 'motorsport',
       'mr', 'news', 'nokia', 'nuclear', 'operators', 'phone', 'plays',
       'premium', 'quest', 'relay', 'return', 'robot', 'robots', 'romen',
       'round', 'ruling', 'said', 'search', 'seed', 'server', 'service',
       'sign', 'site', 'software', 'speegle', 'star', 'study', 'sydney',
       'tansley', 'tuesday', 'tv', 'uk', 'virus', 'wars', 'weather',
       'web', 'women', 'wong', 'yahoo'], dtype='<U18')

In [99]:
show_topics(H1)

['broadband bt million cable line operators news uk',
 'yahoo search google web games tv speegle gamers',
 'mobile nokia flynn said mr romen phone cut',
 'february star kim battlefront game wars return clijsters',
 'asimo robot car robots motorsport left tansley gear',
 'fbi virus mails attachment nuclear inspectors computer agency',
 'site bittorrent apple bloggers legal ruling host server',
 'wong software colour blind mr weather study explore',
 'women seed tuesday sydney plays round quest men',
 'phone relay deaf premium language sign service calls']

In [100]:
W1.shape, H1[:, ind].shape

((20, 10), (10, 78))

### SVD

In [0]:
U, s, V = decomposition.randomized_svd(dtm, 10)

In [0]:
ind = get_all_topic_words(V)

In [105]:
len(ind)

64

In [112]:
get_words_df(V)

,0,1,2,3,4,5,6,7
0,broadband,phone,people,bt,said,million,companies,yahoo
1,broadband,bt,million,line,cable,phone,telephone,companies
2,nokia,mobile,flynn,phone,said,mr,romen,using
3,february,star,kim,clijsters,tennis,return,game,battlefront
4,asimo,robot,car,broadband,left,tansley,motorsport,robots
5,fbi,games,virus,attachment,mails,gamers,yahoo,nuclear
6,site,bittorrent,fbi,apple,virus,attachment,mails,legal
7,software,wong,colour,blind,explore,sake,student,study
8,tuesday,women,sydney,round,plays,seed,hardenne,begins
9,inspectors,nuclear,fbi,information,iaea,relay,deaf,bittorrent


In [107]:
show_topics(H1)

['broadband bt million cable line operators news uk',
 'yahoo search google web games tv speegle gamers',
 'mobile nokia flynn said mr romen phone cut',
 'february star kim battlefront game wars return clijsters',
 'asimo robot car robots motorsport left tansley gear',
 'fbi virus mails attachment nuclear inspectors computer agency',
 'site bittorrent apple bloggers legal ruling host server',
 'wong software colour blind mr weather study explore',
 'women seed tuesday sydney plays round quest men',
 'phone relay deaf premium language sign service calls']

# Apply Topic Modeling on the Whole Dataset 

In [125]:
vectorizer_all = TfidfVectorizer(input='filename', stop_words='english')
dtm_all = vectorizer_all.fit_transform(filenames_all).toarray()
vocab_all = np.array(vectorizer_all.get_feature_names())
dtm_all.shape, len(vocab_all)

((511, 8594), 8594)

### NMF

In [0]:
clf_a = decomposition.NMF(n_components=20, random_state=1)

W1_a = clf_a.fit_transform(dtm_all)
H1_a = clf_a.components_

In [132]:
#embedding to 10 dimensional matrix 
W1_a.shape,H1_a.shape

((511, 20), (20, 8594))

In [0]:
num_top_words=10

def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

In [0]:
def get_all_topic_words(H):
    top_indices = lambda t: {i for i in np.argsort(t)[:-num_top_words-1:-1]}
    topic_indices = [top_indices(t) for t in H]
    return sorted(set.union(*topic_indices))

In [0]:
ind = get_all_topic_words(H1_a)

In [0]:
#ex 
import pandas as pd

def get_all_df(a):
  top_ws = lambda t: [vocab_all[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
  top_w = ([top_ws(t) for t in a])
  df = pd.DataFrame(top_w)
  return df

In [139]:
get_all_df(H1_a)


,0,1,2,3,4,5,6,7,8,9
0,tv,programmes,digital,broadcasting,video,watch,europe,sky,content,iptv
1,open,australian,williams,win,round,champion,safin,grand,davenport,injury
2,microsoft,virus,software,security,windows,spyware,anti,program,programs,users
3,games,game,xbox,play,time,online,sales,video,doom,titles
4,mobile,phone,phones,3g,camera,mobiles,people,operators,vodafone,said
5,broadband,bt,million,people,uk,report,net,connections,signing,fast
6,search,google,desktop,engines,engine,yahoo,web,results,jeeves,people
7,dvd,blu,ray,definition,format,high,dvds,hd,toshiba,technology
8,spam,site,sites,mail,lycos,attacks,screensaver,net,traffic,mails
9,nintendo,ds,handheld,japan,sony,console,gaming,launch,psp,march


In [142]:
W1_a.shape, H1_a[:, ind].shape

((511, 20), (20, 183))

**183 words represent 20 topics**

### SVD

In [0]:
U_a, s_a, V_a = decomposition.randomized_svd(dtm_all, 20)

In [0]:
ind = get_all_topic_words(V_a)

In [148]:
len(ind)

116

In [150]:
get_all_df(V_a)

,0,1,2,3,4,5,6,7,8,9
0,said,mobile,people,music,digital,games,mr,phone,tv,game
1,open,australian,win,game,seed,roddick,match,final,set,second
2,software,search,microsoft,virus,security,windows,spyware,users,mail,spam
3,games,game,nintendo,ds,sony,microsoft,console,dvd,software,gaming
4,music,digital,players,broadband,portable,tv,ipod,apple,media,consumer
5,broadband,games,bt,game,million,ds,nintendo,search,tv,uk
6,search,google,desktop,engines,engine,yahoo,jeeves,web,results,ask
7,dvd,definition,blu,ray,high,technology,dvds,hd,tv,format
8,spam,site,sites,mr,lycos,attacks,said,screensaver,mail,traffic
9,nintendo,ds,spam,sony,japan,dvd,mail,handheld,virus,launch


In [151]:
U_a.shape, V_a.shape

((511, 20), (20, 8594))